Extracción de información. Deberás implementar varias funciones que, recibiendo como 
entrada un string (un post), devuelvan una lista con los resultados obtenidos. 
 
- find_subreddit_mentions(text: str):Permitirá extraer los subreddits mencionados en 
un post. Por ejemplo: “'I\'m cross posting this from /r/cyberlaw, hopefully you guys find it 
as interesting”. Se debe extraer en este caso /r/cyberlaw. En caso de que haya más de uno, 
se deberán extraer todos y guardarlos en una lista. Para ello, se deberá utiliza una única 
expresión regular. 
- url_extraction(text: str) à Permitirá extraer todas las URLs en un post mediante una única 
expresión regular 
- phone_number_extracion(text: str): à Permitirá la extracción de números de teléfono 
mediante una única expresión regular 
- dates_extraction(text: str): à Permitirá la extracción de todas las fechas contenidas en un 
post. 
- code_extraction(text:str): Extracción de código de programación o HTML incluido en un 
post. Permitirá la extracción de todo el código que se incluya en un post. 

Todas  las  funciones  estarán  explicadas  y  detalladas  en  el  notebook  correspondiente, 
incluyendo su implementación final en 5 nuevas funciones en el archivo core.py

<h3>Patrones 're' y desarrollo de código</h3>

- Teniendo en cuenta que ya se ha debido eliminar tanto url's, números de teléfono, fechas, código HTML o menciones a subreddit para la adición de la columna 'post', gran parte del código será reutilizado aunque parcialmente modificado (centrarse en retornar 'matches' más que en eliminarlos).

- Para la extracción del patrón de subreddit nos atenemos a las normas de construcción permitidas para los nombres
```def find_subreddit_mentions(text: str)```

- Para la extracción de números de teléfono tomamos en cuenta las siguientes posibilidades, procurando abarcar las máximas posibilidades: 
```
telefonos: [
    "Contacta al 123-456-7890 para más información.",
    "Nuestro número es (098) 765-4321.",
    "Llámanos al +1 234 567 8901 o al 345.678.9012.",
    "Número internacional: +44 20 7946 0958."
    "De 3 en 3 dígitos sin prefijo: 654 321 890."
]
phone_number_pattern = re.compile(
    r'(\+?\d{1,3}[-.\s]?)?'      # Prefijo
    r'(\(?\d{3}\)?[-.\s]?)?'     # Código de área 
    r'(\d{3}[-.\s]?\d{4})'       # Número de teléfono principal
    r'|(\+?\d{1,3}[-.\s]?)?'     # Prefijo internacional opcional 3 dígitos
    r'(\d{3}[-.\s]\d{3}[-.\s]\d{3})'  # Formato tipo "654 231 235"
)
```

In [ ]:
from typing import List
import re

def find_subreddit_mentions(text: str) -> List[str]|str:
    subreddit_mention_pattern = re.compile(r'/r/[A-Za-z0-9]')
    subreddit_mentions = re.findall(subreddit_mention_pattern, text)
    return subreddit_mentions

def url_extraction(text: str) -> List[str]|str:
    url_pattern = patron_url = re.compile(
    r'(https?://(?:www\.|(?!www))[^\s]+)|'      # URLs con http:// o https://
    r'(www\.[^\s]+)|'                           # URLs que empiezan con www.
    r'([a-zA-Z0-9.-]+\.[a-zA-Z]{2,})'           # URLs sin protocolo
    )
    urls = re.findall(text, url_pattern)
    return urls

def phone_number_extraction(text: str) -> List[str]|str:
    phone_number_pattern = re.compile(
    r'(\+?\d{1,3}[-.\s]?)?'      # Prefijo internacional opcional
    r'(\(?\d{3}\)?[-.\s]?)?'     # Código de área opcional con o sin paréntesis
    r'(\d{3}[-.\s]?\d{4})'       # Número de teléfono principal
    r'|(\+?\d{1,3}[-.\s]?)?'     # Prefijo internacional opcional para números de 3 grupos
    r'(\d{3}[-.\s]\d{3}[-.\s]\d{3})'  # Formato tipo "654 231 235"
    )
    phone_numbers = re.findall(text, phone_number_pattern)
    return phone_numbers
def dates_extraction(text: str) -> List[str]|str:
    dates_pattern = re.compile(r'')
def code_extraction(text: str) -> List[str]|str:
    from bs4 import BeautifulSoup
    plain_text = BeautifulSoup(text, 'html.parser').get_text()
    return plai